<a href="https://colab.research.google.com/github/bukky24/Git-girl-project-2/blob/master/Introduction_to_Regression_Analysis_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np

#Training (891 Entries) & Testing (417 Entries) data
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
all_data = [train_data, test_data]


In [0]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [0]:
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [0]:
def missingdata(data):
    total = data.isnull().sum().sort_values(ascending = False)
    percent = (data.isnull().sum()/data.isnull().count()*100).sort_values(ascending = False)
    ms=pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    ms= ms[ms["Percent"] > 0]
    return ms

In [0]:
missingdata(train_data)

,Total,Percent
Cabin,687,77.104377
Age,177,19.865320
Embarked,2,0.224467


In [0]:
missingdata(test_data)

,Total,Percent
Cabin,327,78.229665
Age,86,20.574163
Fare,1,0.239234


## Filling Missing Values

In [0]:
train_data['Embarked'].fillna(train_data['Embarked'].mode()[0], inplace = True)
test_data['Fare'].fillna(test_data['Fare'].median(), inplace = True)

In [0]:
test_data['Age'].fillna(test_data['Age'].median(), inplace = True)
train_data['Age'].fillna(train_data['Age'].median(), inplace = True)

In [0]:
drop_column = ['Cabin']
train_data.drop(drop_column, axis=1, inplace = True)
test_data.drop(drop_column,axis=1,inplace=True)

In [0]:
print('check the nan value in train data')
print(train_data.isnull().sum())
print('___'*30)
print('check the nan value in test data')
print(test_data.isnull().sum())

check the nan value in train data
PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64
__________________________________________________________________________________________
check the nan value in test data
PassengerId    0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64


**Feature Engineering**

In [0]:
import re
def get_title(name):
    title_search = re.search(' ([A-Za-z]+)\.', name)
  
    if title_search:
        return title_search.group(1)
    return ""
for dataset in all_data:
    dataset['Title'] = dataset['Name'].apply(get_title)

for dataset in all_data:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 
                                                 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
    
print(pd.crosstab(train_data['Title'], train_data['Sex']))


Sex     female  male
Title               
Master       0    40
Miss       185     0
Mr           0   517
Mrs        126     0
Rare         3    20


In [0]:
for dataset in all_data:
    dataset['Age_bin'] = pd.cut(dataset['Age'], bins=[0,12,20,40,120], labels=['Children','Teenage','Adult','Elder'])

In [0]:
for dataset in all_data:
    dataset['Fare_bin'] = pd.cut(dataset['Fare'], bins=[0,7.91,14.45,31,120], labels=['Low_fare','median_fare',
                                                                                      'Average_fare','high_fare'])

In [0]:
traindata=train_data
testdata=test_data
alldata = [traindata, testdata]

In [0]:
for dataset in alldata:
    drop_column = ['Age','Fare','Name','Ticket']
    dataset.drop(drop_column, axis=1, inplace = True)

In [0]:
drop_column = ['PassengerId']
traindata.drop(drop_column, axis=1, inplace = True)

In [0]:
testdata.head(5)

,PassengerId,Pclass,Sex,SibSp,Parch,Embarked,Title,Age_bin,Fare_bin
0,892,3,male,0,0,Q,Mr,Adult,Low_fare
1,893,3,female,1,0,S,Mrs,Elder,Low_fare
2,894,2,male,0,0,Q,Mr,Elder,median_fare
3,895,3,male,0,0,S,Mr,Adult,median_fare
4,896,3,female,1,1,S,Mrs,Adult,median_fare


One_Hot Encoding

In [0]:
traindata = pd.get_dummies(traindata, columns = ["Sex","Title","Age_bin","Embarked","Fare_bin"],
                             prefix=["Sex","Title","Age_type","Em_type","Fare_type"])


In [0]:
testdata = pd.get_dummies(testdata, columns = ["Sex","Title","Age_bin","Embarked","Fare_bin"],
                             prefix=["Sex","Title","Age_type","Em_type","Fare_type"])


In [0]:
testdata.head()

,PassengerId,Pclass,SibSp,Parch,Sex_female,Sex_male,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Rare,Age_type_Children,Age_type_Teenage,Age_type_Adult,Age_type_Elder,Em_type_C,Em_type_Q,Em_type_S,Fare_type_Low_fare,Fare_type_median_fare,Fare_type_Average_fare,Fare_type_high_fare
0,892,3,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,1,0,0,0
1,893,3,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,1,0,0,0
2,894,2,0,0,0,1,0,0,1,0,0,0,0,0,1,0,1,0,0,1,0,0
3,895,3,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,1,0,0
4,896,3,1,1,1,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0


Test/Train Split

In [0]:
from sklearn.model_selection import train_test_split   

X_train = traindata.drop("Survived", axis=1)
Y_train = traindata["Survived"]
X_test  = testdata.drop("PassengerId", axis=1).copy()
train_X, test_X, train_y, test_y = train_test_split(
    X_train, Y_train, test_size=0.20,random_state=0)

In [85]:
X_test.shape

(418, 21)

# Logistic Regression

In [0]:
lr = LogisticRegression(solver = 'lbfgs')
lr.fit(train_X, train_y)
predictions = lr.predict(test_X)

## Testing the model accuracy using F1 Score

In [88]:
from sklearn.metrics import f1_score
lr = LogisticRegression(solver ='lbfgs')
lr.fit(train_X, train_y)
predictions = lr.predict(test_X)
fscore = f1_score(test_y, predictions)
print(fscore)

0.762589928057554


# Cross-Validation

In [75]:
from sklearn.model_selection import cross_val_score

lr = LogisticRegression(solver = 'lbfgs')
scores = cross_val_score(lr, X_train,Y_train, cv=10)
scores.sort()
accuracy = scores.mean()

print(scores)
print(accuracy)

[0.78651685 0.78651685 0.79775281 0.80898876 0.81111111 0.82022472
 0.82222222 0.83146067 0.86363636 0.86516854]
0.8193598910452844


# **Prediction**

In [89]:
lr = LogisticRegression(solver = 'lbfgs')
lr.fit(train_X, train_y)
predictions = lr.predict(X_test)
print (predictions)

[0 0 0 0 1 0 1 0 1 0 0 0 1 0 1 1 0 0 1 1 0 1 1 0 1 0 1 0 0 0 0 0 1 1 0 0 1
 1 0 0 0 0 0 1 1 0 0 0 1 1 0 0 1 1 0 0 0 0 0 1 0 0 0 1 1 1 1 0 0 1 1 0 1 0
 1 0 0 1 0 1 1 0 0 0 0 0 1 1 1 1 1 0 1 0 0 0 1 0 1 0 1 0 0 0 1 0 0 0 0 0 0
 1 1 1 1 0 0 1 1 1 1 0 1 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0
 0 0 1 0 0 1 0 0 1 1 0 1 1 1 1 0 0 1 0 0 1 1 0 0 0 0 0 1 1 0 1 1 0 0 1 0 1
 0 1 0 0 0 0 0 1 0 1 0 1 1 0 1 1 1 0 1 0 0 1 0 1 0 0 0 0 1 0 0 1 0 1 0 1 0
 1 0 1 1 0 1 0 0 0 1 0 0 0 0 0 0 1 1 1 1 0 0 0 0 1 0 1 1 1 0 0 0 0 0 0 0 1
 0 0 0 1 1 0 0 0 0 1 0 0 0 1 1 0 1 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 1 0 0 0 0
 1 0 0 0 0 0 0 0 1 1 0 1 0 1 0 0 0 1 1 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0
 1 0 0 0 0 0 1 0 0 0 1 0 1 0 1 0 1 1 0 0 0 1 0 1 0 0 1 0 1 1 0 1 0 0 1 1 0
 0 1 0 0 1 1 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 1 1 0 0 1 0 1 0 0 1 0 1 1 0 0 0
 0 1 1 1 1 1 0 1 0 0 1]


# **Saving to Pandas** **Dataframe**

In [0]:
pd_dataframe = pd.DataFrame({
        "PassengerId": testdata["PassengerId"],
        "Survived": predictions})